In [8]:
import numpy as np
import pandas as pd
from pathlib import Path
import re

# import sqlite3
import pymysql
from sqlalchemy import create_engine

In [2]:
class Excel2DB:
    def __init__(self) -> None:
        self.data_path = Path('./延误指数/')
        
#         self.conn = sqlite3.connect('延迟指数.db')
        self.conn = pymysql.connect(
            host='localhost',
            user='root',
            port=3306,
            password='Senkita.cc/',
            db='delay_index',
            charset='utf8mb4'
        )
        self.cur = self.conn.cursor() # 游标
        
        self.date_sql = self.__date_sql_concat()
        
    def __del__(self) -> None:
        self.conn.commit() # 事务提交
        self.cur.close() # 游标关闭
        self.conn.close() # 连接关闭
        
    # 生成日期字段
    def __date_sql_concat(self) -> str:
        date_sql_list = [
#             '"%02d%02d" REAL' % (d.month, d.day)
            'T%02d%02d FLOAT' % (d.month, d.day)
            for d in pd.date_range(
                start='2020-09-01',
                end='2020-11-30',
                freq='D'
            )
        ]
        return ', '.join(date_sql_list)

    # 建表函数
    def __create_table(self, table_name: str) -> None:
#         self.cur.execute('''
#             CREATE TABLE IF NOT EXISTS {}(
#                 time PRIMARY KEY,
        self.cur.execute('''
            CREATE TABLE IF NOT EXISTS {}(
                time INT PRIMARY KEY UNIQUE KEY,
                {}
            );
            '''.format(
                table_name,
                self.date_sql
            )
        )

    # 添加数据
    def __insert_data(self, table_name: str, data_list: list) -> None:
#         self.cur.execute('''
#             INSERT OR IGNORE INTO {}(
        self.cur.execute('''
            INSERT INTO {}(
                time,
                {}
            ) VALUES ({});
            '''.format(
                table_name,
#                 self.date_sql.replace(' REAL', ''),
                self.date_sql.replace(' FLOAT', ''),
                ', '.join(repr(e) for e in data_list)
            )
        )
    
    def run(self) -> None:
        pattern = re.compile(r'[.|\-|,]') # 正则筛选出表名不支持字符

        for file in self.data_path.iterdir():
            if file.suffix == '.xls':
                table_name = pattern.sub('_', file.stem.split('.')[1])
                self.__create_table(table_name)

                df = pd.read_excel(Path.joinpath(self.data_path, file.name))
                df = df.drop(['Unnamed: 0'], axis=1)
                for row in range(df.shape[0]):
                    self.__insert_data(table_name, df.loc[row].tolist())

In [12]:
class HandleDB:
    def __init__(self) -> None:
        self.data_path = Path('./延误指数/')
        
#         self.conn = sqlite3.connect('延迟指数.db')
        self.conn = pymysql.connect(
            host='localhost',
            user='root',
            port=3306,
            password='Senkita.cc/',
            db='delay_index',
            charset='utf8mb4'
        )
        self.cur = self.conn.cursor() # 游标
            
        self.engine = create_engine('mysql+pymysql://root:Senkita.cc/@localhost:3306/delay_index?charset=utf8mb4')
        
        self.route_dict = self.__grouping_table_name()

    def __del__(self) -> None:
        try:
            self.conn.commit() # 事务提交
            self.cur.close() # 游标关闭
            self.conn.close() # 连接关闭
        
    # 获取表名
    def __get_table_name(self) -> list:
#         self.cur.execute('''
#             SELECT name FROM sqlite_master WHERE type="table" GROUP BY name;
        self.cur.execute('''
            SHOW TABLES;
        ''')
        return self.cur.fetchall()

    # 表名分组
    def __grouping_table_name(self) -> dict:
        route_dict = {}
        for r in self.__get_table_name():
            road_section = r[0]
            road_name_list = road_section.split('_')
            road_name = road_name_list[0]
            direction = road_name_list[-1]

            route_dict.setdefault(road_name+direction, []).append(road_section)
        return route_dict

    def get_mean(self, scence: str) -> None:
        for i in self.route_dict.keys():
            if scence == 'all_day':
                union_sql = ' UNION '.join(['(SELECT * FROM {})'.format(route) for route in self.route_dict[i]])
                file_path = '均值'
            elif scence == 'rush_hour':
                union_sql = ' UNION '.join(['(SELECT * FROM {} WHERE (time BETWEEN 700 AND 900) OR (time BETWEEN 1700 AND 1900))'.format(route) for route in self.route_dict[i]])
                file_path = '高峰均值'
            elif scence == 'morning_rush':
                union_sql = ' UNION '.join(['(SELECT * FROM {} WHERE (time BETWEEN 700 AND 900))'.format(route) for route in self.route_dict[i]])
                file_path = '早高峰均值'
            else:
                union_sql = ' UNION '.join(['(SELECT * FROM {} WHERE (time BETWEEN 1700 AND 1900))'.format(route) for route in self.route_dict[i]])
                file_path = '晚高峰均值'
            
            df = pd.read_sql(
                sql=union_sql,
                con=self.engine
            )
            df = df.drop(['time'], axis=1)
            df = df.replace(0, np.nan)

            df.mean(
                axis=0,
                skipna=True
            ).to_frame().reset_index().T.to_excel(
                './{}/{}.xlsx'.format(
                    file_path,
                    i
                ),
                index=False
            )

In [6]:
def main() -> None:
    Excel2DB().run() # Excel转数据库
    
    hd = HandleDB()
    hd.get_mean(scence='all_day') # 全天
    hd.get_mean(scence='rush_hour') # 全高峰
    hd.get_mean(scence='morning_rush') # 早高峰
    hd.get_mean(scence='evening_peak') # 晚高峰

In [13]:
if __name__ == '__main__':
    main()

Exception ignored in: <function HandleDB.__del__ at 0x7ff644d1d950>
Traceback (most recent call last):
  File "<ipython-input-5-b2d42b022ad7>", line 25, in __del__
AttributeError: 'HandleDB' object has no attribute 'conn'
